# Imports and Setup

In [1]:
!pip uninstall -y flwr ray
!pip install -U "flwr[simulation]" ray
!pip install pyannote.metrics
!pip install pyannote.core
!pip install speechbrain
!pip install datasets

Found existing installation: flwr 1.15.2
Uninstalling flwr-1.15.2:
  Successfully uninstalled flwr-1.15.2
Found existing installation: ray 2.31.0
Uninstalling ray-2.31.0:
  Successfully uninstalled ray-2.31.0
  Using cached ray-2.43.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached flwr-1.15.2-py3-none-any.whl.metadata (15 kB)
  Using cached ray-2.31.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (13 kB)
Using cached ray-2.31.0-cp311-cp311-manylinux2014_x86_64.whl (66.7 MB)
Using cached flwr-1.15.2-py3-none-any.whl (531 kB)


In [2]:
# Step 1: Load AMI dataset and group by meeting ID
from datasets import load_dataset

dataset = load_dataset("edinburghcstr/ami", "ihm")

def group_by_meeting(dataset_split):
    grouped = {}
    for sample in dataset_split:
        meeting_id = sample["meeting_id"]
        grouped.setdefault(meeting_id, []).append(sample)
    return grouped

grouped_train = group_by_meeting(dataset["train"].select(range(5000)))
grouped_validation = group_by_meeting(dataset["validation"].select(range(5000)))
grouped_test = group_by_meeting(dataset["test"].select(range(5000)))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
!pip install speechbrain

In [4]:
import os
import pickle
from collections import OrderedDict
from typing import List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import librosa
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset, random_split
import flwr as fl
from flwr.client import NumPyClient
from flwr.common import Context
from pyannote.core import Segment, Annotation
from pyannote.metrics.diarization import DiarizationErrorRate
from speechbrain.pretrained import EncoderClassifier

/usr/local/lib/python3.11/dist-packages/pyannote/core/notebook.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cm = get_cmap("Set1")
<ipython-input-4-0ce21e2d0acc>:21: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
from speechbrain.pretrained import EncoderClassifier
from torch.optim import Adam
from pyannote.metrics.diarization import DiarizationErrorRate
from scipy.stats import mode

torch.cuda.empty_cache()

# **Set device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Load Pretrained Speaker Encoder (ECAPA-TDNN)**
speaker_encoder = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_models/spkrec-ecapa",
    run_opts={"device": device}
).to(device)

# **Freeze the speaker encoder**
for param in speaker_encoder.parameters():
    param.requires_grad = False

# **Function to extract speaker embeddings**
def extract_speaker_embedding(features):
    """Extracts speaker embeddings using ECAPA-TDNN."""
    with torch.no_grad():
        embeddings = speaker_encoder.encode_batch(features)
        return embeddings.squeeze(1)  # Remove unnecessary dimension

# **Power-Set Encoding Function**
def power_set_encoding(labels):
    """Encodes multi-label speaker activation into a single integer using power-set encoding."""
    return sum([l * (2 ** i) for i, l in enumerate(labels) if l in [0, 1]])

# **Overlapping Speech Simulation**
def simulate_overlap(grouped_data, num_speakers=2, duration=10, sampling_rate=16000):
    overlapping_segments = []
    labels = []
    for meeting_id, samples in grouped_data.items():
        speakers = {sample["speaker_id"]: [] for sample in samples}
        for sample in samples:
            speakers[sample["speaker_id"]].append(sample)
        if len(speakers) < num_speakers:
            continue
        for _ in range(50):
            chosen_speakers = random.sample(list(speakers.keys()), num_speakers)
            combined_signal = None
            speaker_label = [0] * len(speakers)
            for speaker_id in chosen_speakers:
                sample = random.choice(speakers[speaker_id])
                signal = sample["audio"]["array"]
                if len(signal) > duration * sampling_rate:
                    signal = signal[: duration * sampling_rate]
                else:
                    signal = np.pad(signal, (0, duration * sampling_rate - len(signal)), mode="constant")
                if combined_signal is None:
                    combined_signal = signal
                else:
                    combined_signal += signal
                speaker_label[list(speakers.keys()).index(speaker_id)] = 1
            if combined_signal is not None and np.max(np.abs(combined_signal)) > 0:
                combined_signal = combined_signal / np.max(np.abs(combined_signal))
            else:
                continue
            overlapping_segments.append(combined_signal)
            labels.append(power_set_encoding(speaker_label))

    # ✅ Debug: Print the first few labels to check if overlapping speech is simulated correctly
    print("🔹 Sample labels from train set (Power-Set Encoded):")
    for i in range(min(10, len(labels))):  # Print up to 10 samples
        print(labels[i])

    return overlapping_segments, labels


# **Feature Extraction (Log-Mel Spectrograms)**
def extract_features_batch(segments, sr=16000, n_mels=80, win_length=0.025, hop_length=0.01, max_frames=None):
    features = []
    win_length_samples = int(win_length * sr)
    hop_length_samples = int(hop_length * sr)

    for segment in segments:
        mel_spec = librosa.feature.melspectrogram(
            y=segment, sr=sr, n_mels=n_mels, win_length=win_length_samples, hop_length=hop_length_samples
        )
        log_mel = librosa.power_to_db(mel_spec)

        if max_frames:
            log_mel = log_mel[:, :max_frames] if log_mel.shape[1] > max_frames else np.pad(
                log_mel, ((0, 0), (0, max_frames - log_mel.shape[1])), mode="constant"
            )

        features.append(log_mel.T)

    features = np.array(features, dtype=np.float32)

    return torch.tensor(features, dtype=torch.float32)  # ✅ Corrected tensor conversion

class OverlappingSpeechDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        # Ensure label shape is [time] to match outputs
        label = label.expand(feature.shape[0])

        return feature, label

scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Scaling

/usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
/usr/local/lib/python3.11/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seria

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
from speechbrain.pretrained import EncoderClassifier
from torch.optim import Adam
from pyannote.metrics.diarization import DiarizationErrorRate
from scipy.stats import mode

torch.cuda.empty_cache()

# **Set device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Load Pretrained Speaker Encoder (ECAPA-TDNN)**
speaker_encoder = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_models/spkrec-ecapa",
    run_opts={"device": device}
).to(device)

# **Freeze the speaker encoder**
for param in speaker_encoder.parameters():
    param.requires_grad = False

# **Function to extract speaker embeddings**
def extract_speaker_embedding(features):
    """Extracts speaker embeddings using ECAPA-TDNN."""
    with torch.no_grad():
        embeddings = speaker_encoder.encode_batch(features)
        return embeddings.squeeze(1)  # Remove unnecessary dimension

# **Power-Set Encoding Function**
def power_set_encoding(labels):
    """Encodes multi-label speaker activation into a single integer using power-set encoding."""
    return sum([l * (2 ** i) for i, l in enumerate(labels) if l in [0, 1]])

# **Overlapping Speech Simulation**
def simulate_overlap(grouped_data, num_speakers=2, duration=10, sampling_rate=16000):
    overlapping_segments = []
    labels = []
    for meeting_id, samples in grouped_data.items():
        speakers = {sample["speaker_id"]: [] for sample in samples}
        for sample in samples:
            speakers[sample["speaker_id"]].append(sample)
        if len(speakers) < num_speakers:
            continue
        for _ in range(50):
            chosen_speakers = random.sample(list(speakers.keys()), num_speakers)
            combined_signal = None
            speaker_label = [0] * len(speakers)
            for speaker_id in chosen_speakers:
                sample = random.choice(speakers[speaker_id])
                signal = sample["audio"]["array"]
                if len(signal) > duration * sampling_rate:
                    signal = signal[: duration * sampling_rate]
                else:
                    signal = np.pad(signal, (0, duration * sampling_rate - len(signal)), mode="constant")
                if combined_signal is None:
                    combined_signal = signal
                else:
                    combined_signal += signal
                speaker_label[list(speakers.keys()).index(speaker_id)] = 1
            if combined_signal is not None and np.max(np.abs(combined_signal)) > 0:
                combined_signal = combined_signal / np.max(np.abs(combined_signal))
            else:
                continue
            overlapping_segments.append(combined_signal)
            labels.append(power_set_encoding(speaker_label))

    # ✅ Debug: Print the first few labels to check if overlapping speech is simulated correctly
    print("🔹 Sample labels from train set (Power-Set Encoded):")
    for i in range(min(10, len(labels))):  # Print up to 10 samples
        print(labels[i])

    return overlapping_segments, labels


# **Feature Extraction (Log-Mel Spectrograms)**
def extract_features_batch(segments, sr=16000, n_mels=80, win_length=0.025, hop_length=0.01, max_frames=None):
    features = []
    win_length_samples = int(win_length * sr)
    hop_length_samples = int(hop_length * sr)

    for segment in segments:
        mel_spec = librosa.feature.melspectrogram(
            y=segment, sr=sr, n_mels=n_mels, win_length=win_length_samples, hop_length=hop_length_samples
        )
        log_mel = librosa.power_to_db(mel_spec)

        if max_frames:
            log_mel = log_mel[:, :max_frames] if log_mel.shape[1] > max_frames else np.pad(
                log_mel, ((0, 0), (0, max_frames - log_mel.shape[1])), mode="constant"
            )

        features.append(log_mel.T)

    features = np.array(features, dtype=np.float32)

    return torch.tensor(features, dtype=torch.float32)  # ✅ Corrected tensor conversion

class OverlappingSpeechDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        # Ensure label shape is [time] to match outputs
        label = label.expand(feature.shape[0])

        return feature, label

scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Scaling

# **Training Execution**
num_speakers = 5

# Simulate overlapping speech for training
train_segments, train_labels = simulate_overlap(grouped_train)
val_segments, val_labels = simulate_overlap(grouped_validation)
test_segments, test_labels = simulate_overlap(grouped_test)

from collections import Counter

print("🔹 Unique Labels in Training Set:", set(train_labels))
print("🔹 Label Distribution:")
label_counts = Counter(train_labels)
for label, count in label_counts.items():
    binary_repr = bin(label)[2:].zfill(num_speakers)  # Convert to binary
    print(f"Label: {label}, Binary: {binary_repr}, Count: {count}")

unique_labels = sorted(set(train_labels + val_labels))  # ✅ Include val_labels
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
index_to_label = {idx: label for label, idx in label_to_index.items()}

print(f"🔹 Label Mapping: {label_to_index}")

num_classes = len(unique_labels)  # ✅ Actual number of classes

# Apply mapping to training and validation labels
train_labels = [label_to_index[label] for label in train_labels]
val_labels = [label_to_index[label] for label in val_labels]

print(f"🔹 Mapped Training Labels: {train_labels[:10]}")  # Debugging


max_frames = int((10 / 0.01))  # 10s duration, 0.01 hop length (10s / 10ms)
train_features = extract_features_batch(train_segments, max_frames=max_frames)
val_features = extract_features_batch(val_segments, max_frames=max_frames)
test_features = extract_features_batch(test_segments, max_frames=max_frames)

train_loader = DataLoader(OverlappingSpeechDataset(train_features, train_labels), batch_size=16, shuffle=True)
val_loader = DataLoader(OverlappingSpeechDataset(val_features, val_labels), batch_size=16, shuffle=False)
test_loader = DataLoader(OverlappingSpeechDataset(test_features, test_labels), batch_size=16, shuffle=False)

<ipython-input-6-0b7e27721769>:122: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Scaling


🔹 Sample labels from train set (Power-Set Encoded):
12
5
17
24
10
10
12
10
20
10
🔹 Sample labels from train set (Power-Set Encoded):
12
10
3
10
9
9
5
10
9
10
🔹 Sample labels from train set (Power-Set Encoded):
6
10
3
5
9
9
5
10
3
12
🔹 Unique Labels in Training Set: {3, 5, 6, 9, 10, 12, 17, 18, 20, 24}
🔹 Label Distribution:
Label: 12, Binary: 01100, Count: 22
Label: 5, Binary: 00101, Count: 23
Label: 17, Binary: 10001, Count: 13
Label: 24, Binary: 11000, Count: 15
Label: 10, Binary: 01010, Count: 19
Label: 20, Binary: 10100, Count: 16
Label: 6, Binary: 00110, Count: 30
Label: 9, Binary: 01001, Count: 26
Label: 18, Binary: 10010, Count: 13
Label: 3, Binary: 00011, Count: 23
🔹 Label Mapping: {3: 0, 5: 1, 6: 2, 9: 3, 10: 4, 12: 5, 17: 6, 18: 7, 20: 8, 24: 9}
🔹 Mapped Training Labels: [5, 1, 6, 9, 4, 4, 5, 4, 8, 4]


In [7]:
# import logging
# import time
# import flwr as fl

# import importlib
# import ray
# importlib.reload(ray)

# from speechbrain.pretrained import EncoderClassifier

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import librosa
# import numpy as np
# import random
# from torch.utils.data import Dataset, DataLoader
# from speechbrain.pretrained import EncoderClassifier
# from torch.optim import Adam
# from pyannote.metrics.diarization import DiarizationErrorRate
# from scipy.stats import mode

# torch.cuda.empty_cache()

# # **Set device**
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # **Load Pretrained Speaker Encoder (ECAPA-TDNN)**
# speaker_encoder = EncoderClassifier.from_hparams(
#     source="speechbrain/spkrec-ecapa-voxceleb",
#     savedir="pretrained_models/spkrec-ecapa",
#     run_opts={"device": device}
# ).to(device)

# # **Freeze the speaker encoder**
# for param in speaker_encoder.parameters():
#     param.requires_grad = False

# # **Function to extract speaker embeddings**
# def extract_speaker_embedding(features):
#     """Extracts speaker embeddings using ECAPA-TDNN."""
#     with torch.no_grad():
#         embeddings = speaker_encoder.encode_batch(features)
#         return embeddings.squeeze(1)  # Remove unnecessary dimension

# # **Power-Set Encoding Function**
# def power_set_encoding(labels):
#     """Encodes multi-label speaker activation into a single integer using power-set encoding."""
#     return sum([l * (2 ** i) for i, l in enumerate(labels) if l in [0, 1]])

# # **Overlapping Speech Simulation**
# def simulate_overlap(grouped_data, num_speakers=2, duration=10, sampling_rate=16000):
#     overlapping_segments = []
#     labels = []
#     for meeting_id, samples in grouped_data.items():
#         speakers = {sample["speaker_id"]: [] for sample in samples}
#         for sample in samples:
#             speakers[sample["speaker_id"]].append(sample)
#         if len(speakers) < num_speakers:
#             continue
#         for _ in range(50):
#             chosen_speakers = random.sample(list(speakers.keys()), num_speakers)
#             combined_signal = None
#             speaker_label = [0] * len(speakers)
#             for speaker_id in chosen_speakers:
#                 sample = random.choice(speakers[speaker_id])
#                 signal = sample["audio"]["array"]
#                 if len(signal) > duration * sampling_rate:
#                     signal = signal[: duration * sampling_rate]
#                 else:
#                     signal = np.pad(signal, (0, duration * sampling_rate - len(signal)), mode="constant")
#                 if combined_signal is None:
#                     combined_signal = signal
#                 else:
#                     combined_signal += signal
#                 speaker_label[list(speakers.keys()).index(speaker_id)] = 1
#             if combined_signal is not None and np.max(np.abs(combined_signal)) > 0:
#                 combined_signal = combined_signal / np.max(np.abs(combined_signal))
#             else:
#                 continue
#             overlapping_segments.append(combined_signal)
#             labels.append(power_set_encoding(speaker_label))

#     # ✅ Debug: Print the first few labels to check if overlapping speech is simulated correctly
#     print("🔹 Sample labels from train set (Power-Set Encoded):")
#     for i in range(min(10, len(labels))):  # Print up to 10 samples
#         print(labels[i])

#     return overlapping_segments, labels


# # **Feature Extraction (Log-Mel Spectrograms)**
# def extract_features_batch(segments, sr=16000, n_mels=80, win_length=0.025, hop_length=0.01, max_frames=None):
#     features = []
#     win_length_samples = int(win_length * sr)
#     hop_length_samples = int(hop_length * sr)

#     for segment in segments:
#         mel_spec = librosa.feature.melspectrogram(
#             y=segment, sr=sr, n_mels=n_mels, win_length=win_length_samples, hop_length=hop_length_samples
#         )
#         log_mel = librosa.power_to_db(mel_spec)

#         if max_frames:
#             log_mel = log_mel[:, :max_frames] if log_mel.shape[1] > max_frames else np.pad(
#                 log_mel, ((0, 0), (0, max_frames - log_mel.shape[1])), mode="constant"
#             )

#         features.append(log_mel.T)

#     features = np.array(features, dtype=np.float32)

#     return torch.tensor(features, dtype=torch.float32)  # ✅ Corrected tensor conversion

# class OverlappingSpeechDataset(Dataset):
#     def __init__(self, features, labels):
#         self.features = features
#         self.labels = labels

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         feature = torch.tensor(self.features[idx], dtype=torch.float32)
#         label = torch.tensor(self.labels[idx], dtype=torch.long)

#         return feature, label

# scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Scaling

# # **Training Execution**
# num_speakers = 5

# # Simulate overlapping speech for training
# train_segments, train_labels = simulate_overlap(grouped_train)
# val_segments, val_labels = simulate_overlap(grouped_validation)
# test_segments, test_labels = simulate_overlap(grouped_test)

# from collections import Counter

# print("🔹 Unique Labels in Training Set:", set(train_labels))
# print("🔹 Label Distribution:")
# label_counts = Counter(train_labels)
# for label, count in label_counts.items():
#     binary_repr = bin(label)[2:].zfill(num_speakers)  # Convert to binary
#     print(f"Label: {label}, Binary: {binary_repr}, Count: {count}")

# unique_labels = sorted(set(train_labels + val_labels))  # ✅ Include val_labels
# label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
# index_to_label = {idx: label for label, idx in label_to_index.items()}

# print(f"🔹 Label Mapping: {label_to_index}")

# num_classes = len(unique_labels)  # ✅ Actual number of classes

# # Apply mapping to training and validation labels
# train_labels = [label_to_index[label] for label in train_labels]
# val_labels = [label_to_index[label] for label in val_labels]

# print(f"🔹 Mapped Training Labels: {train_labels[:10]}")  # Debugging

# from torch.nn.utils.rnn import pad_sequence

# def collate_fn_pad(batch):
#     """Pads variable-length sequences to the longest in the batch, ensuring consistent batch shapes."""
#     batch = [b for b in batch if b is not None]  # Remove None samples

#     if len(batch) == 0:
#         return None, None  # Return empty batch if all are invalid

#     features, labels = zip(*batch)

#     # ✅ Pad sequences to the maximum length in this batch
#     features_padded = pad_sequence(features, batch_first=True, padding_value=0)

#     # ✅ Keep labels as scalars (no expansion)
#     labels = torch.tensor(labels, dtype=torch.long)

#     return features_padded, labels


# max_frames = int((10 / 0.01))  # 10s duration, 0.01 hop length (10s / 10ms)
# train_features = extract_features_batch(train_segments, max_frames=max_frames)
# val_features = extract_features_batch(val_segments, max_frames=max_frames)
# test_features = extract_features_batch(test_segments, max_frames=max_frames)

# train_loader = DataLoader(
#     OverlappingSpeechDataset(train_features, train_labels),
#     batch_size=16,
#     shuffle=True,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# val_loader = DataLoader(
#     OverlappingSpeechDataset(val_features, val_labels),
#     batch_size=16,
#     shuffle=False,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# test_loader = DataLoader(
#     OverlappingSpeechDataset(test_features, test_labels),
#     batch_size=16,
#     shuffle=False,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # ✅ Define `speaker_encoder` correctly
# speaker_encoder = EncoderClassifier.from_hparams(
#     source="speechbrain/spkrec-ecapa-voxceleb",
#     savedir="pretrained_models/spkrec-ecapa",
#     run_opts={"device": DEVICE}
# ).to(DEVICE)

# # **SEND Model**
# class SpeechEncoder(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers):
#         super().__init__()
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)

#     def forward(self, x):
#         x, _ = self.lstm(x)
#         return x

# class SEND(nn.Module):
#     def __init__(self, input_dim, speaker_dim, hidden_dim, num_layers, num_speakers):
#         super().__init__()
#         self.speech_encoder = SpeechEncoder(input_dim, hidden_dim, num_layers)
#         self.speaker_encoder = speaker_encoder.encode_batch

#         self.feature_projection = nn.Linear(hidden_dim * 2, hidden_dim * 2)
#         self.context_independent = nn.Linear(hidden_dim * 2, num_speakers)
#         self.context_dependent = nn.TransformerEncoder(nn.TransformerEncoderLayer(hidden_dim * 2, 4), num_layers)
#         self.match_projection = nn.Linear(hidden_dim * 2, num_speakers)
#         self.post_net = nn.Linear(hidden_dim * 2, 2 ** num_speakers)  # Ensure input matches hidden_dim * 2

#     def forward(self, speech):
#         # Ensure correct shape
#         if speech.dim() == 4:
#             speech = speech.squeeze(1)

#         speech_features = self.speech_encoder(speech)
#         # print(f"🔹 Speech features shape after LSTM: {speech_features.shape}")  # [16, 29, 512]

#         speech_features = self.feature_projection(speech_features)
#         # print(f"🔹 Projected speech features shape: {speech_features.shape}")  # [16, 29, 512]

#         # **Apply mean pooling across time dimension (sequence length 29)**
#         pooled_features = torch.mean(speech_features, dim=1)  # [16, 512]
#         # print(f"🔹 Pooled features shape: {pooled_features.shape}")  # [16, 512]

#         # Ensure output matches batch size
#         output = self.post_net(pooled_features)  # [16, num_classes]
#         # print(f"🔹 Model output shape: {output.shape}")  # Should be [16, 2**num_speakers]

#         return output

# model = SEND(input_dim=80, speaker_dim=512, hidden_dim=256, num_layers=4, num_speakers=num_speakers).to(DEVICE)

# class FLClient(NumPyClient):
#     def __init__(self, model, train_loader, test_loader):
#         os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # ✅ Force CPU globally
#         self.device = torch.device("cpu")  # ✅ Force CPU for PyTorch
#         logging.info(f"Using device: {self.device}")
#         self.model = model.to(self.device)  # ✅ Move model to CPU
#         self.train_loader = train_loader
#         self.test_loader = test_loader
#         self.optimizer = optim.Adam(model.parameters(), lr=0.001)
#         self.criterion = nn.CrossEntropyLoss()

#     def get_parameters(self, config=None):
#         return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

#     def set_parameters(self, parameters):
#         state_dict = {k: torch.tensor(v).to(self.device) for k, v in zip(self.model.state_dict().keys(), parameters)}
#         self.model.load_state_dict(state_dict)
#         self.model.to(self.device)  # Ensure model is on the right device

#     def fit(self, parameters, config):
#       self.set_parameters(parameters)
#       self.model.train()

#       logging.info(f"Training on {self.device}...")
#       for epoch in range(2):
#           for batch_idx, (features, labels) in enumerate(self.train_loader):
#               features, labels = features.to(self.device), labels.to(self.device)

#               # 🔹 Debug shape mismatch
#               print(f"Batch {batch_idx} - Feature shape: {features.shape}, Label shape: {labels.shape}")

#               # 🔹 Print sample data for debugging
#               print(f"Sample labels: {labels[:5]}")  # Check label values


#               self.optimizer.zero_grad()
#               outputs = self.model(features)

#               # 🔹 Ensure labels match batch size
#               labels = labels.view(-1)  # Flatten labels if needed
#               print(f"Fixed Labels Shape: {labels.shape}")  # Should match outputs.shape[0]
#               print(f"Outputs shape: {outputs.shape}, Labels shape: {labels.shape}")

#               loss = self.criterion(outputs, labels.long())
#               loss.backward()
#               self.optimizer.step()

#       return self.get_parameters(), len(self.train_loader.dataset), {}


#     def evaluate(self, parameters, config):
#         self.set_parameters(parameters)
#         self.model.eval()

#         total_loss = 0
#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for batch_idx, (features, labels) in enumerate(self.test_loader):
#                 features, labels = features.to(self.device), labels.to(self.device)
#                 outputs = self.model(features)

#                 labels = labels.long()
#                 loss = self.criterion(outputs, labels)
#                 total_loss += loss.item()
#                 preds = torch.argmax(outputs, dim=-1)
#                 correct += (preds == labels).sum().item()
#                 total += labels.numel()

#                 # ✅ Add debug logging for evaluation
#                 logging.info(f"🔹 Evaluation Batch {batch_idx}")
#                 logging.info(f"🔹 Predictions: {preds[:5]}")
#                 logging.info(f"🔹 Ground Truth: {labels[:5]}")
#                 logging.info(f"🔹 Batch Loss: {loss.item()}")

#         accuracy = correct / total if total > 0 else 0.0
#         logging.info(f"✅ Evaluation Results - Loss: {total_loss / len(self.test_loader):.4f}, Accuracy: {accuracy:.2%}")
#         return total_loss / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}

# # ✅ Custom Strategy for Evaluation Debugging
# class PrintEvaluateStrategy(fl.server.strategy.FedAvg):
#     def aggregate_evaluate(self, rnd: int, results, failures):
#         aggregated = super().aggregate_evaluate(rnd, results, failures)
#         if aggregated is None:
#             logging.info(f"[ROUND {rnd}] No evaluation results")
#             return aggregated
#         if isinstance(aggregated, tuple):
#             if len(aggregated) == 3:
#                 loss, num_examples, metrics = aggregated
#             elif len(aggregated) == 2:
#                 loss, num_examples = aggregated
#                 num_examples = num_examples if isinstance(num_examples, int) else "unknown"
#                 metrics = {}
#             else:
#                 loss, num_examples, metrics = aggregated, "unknown", {}
#         else:
#             logging.info(f"[ROUND {rnd}] Aggregated evaluation: {aggregated}")
#             return aggregated

#         accuracy = metrics.get("accuracy", None)
#         accuracy_str = f"{accuracy:.2%}" if accuracy is not None else "N/A"
#         logging.info(f"[ROUND {rnd}] Evaluation results: Loss: {loss:.4f}, Accuracy: {accuracy_str} on {num_examples} examples")
#         return aggregated

# logging.basicConfig(level=logging.INFO)
# strategy = PrintEvaluateStrategy(min_fit_clients=4, min_available_clients=4)

# for features, labels in train_loader:
#     print(f"Feature shape: {features.shape}, Label shape: {labels.shape}")
#     break  # Only check first batch


# fl.simulation.start_simulation(
#     client_fn=lambda ctx: FLClient(model, train_loader, test_loader),
#     num_clients=4,
#     config=fl.server.ServerConfig(num_rounds=2),
#     strategy=fl.server.strategy.FedAvg(),
#     ray_init_args={
#         "num_cpus": 4,  # ✅ Only use CPU
#         "include_dashboard": False,
#         "ignore_reinit_error": True,
#     },
#     client_resources={"num_cpus": 1}  # ✅ No GPU needed
# )

In [8]:
# import logging
# import time
# import flwr as fl

# import importlib
# import ray
# importlib.reload(ray)

# from speechbrain.pretrained import EncoderClassifier

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import librosa
# import numpy as np
# import random
# from torch.utils.data import Dataset, DataLoader
# from speechbrain.pretrained import EncoderClassifier
# from torch.optim import Adam
# from pyannote.metrics.diarization import DiarizationErrorRate
# from scipy.stats import mode

# torch.cuda.empty_cache()

# # **Set device**
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # **Load Pretrained Speaker Encoder (ECAPA-TDNN)**
# speaker_encoder = EncoderClassifier.from_hparams(
#     source="speechbrain/spkrec-ecapa-voxceleb",
#     savedir="pretrained_models/spkrec-ecapa",
#     run_opts={"device": device}
# ).to(device)

# # **Freeze the speaker encoder**
# for param in speaker_encoder.parameters():
#     param.requires_grad = False

# # **Function to extract speaker embeddings**
# def extract_speaker_embedding(features):
#     """Extracts speaker embeddings using ECAPA-TDNN."""
#     with torch.no_grad():
#         embeddings = speaker_encoder.encode_batch(features)
#         return embeddings.squeeze(1)  # Remove unnecessary dimension

# # **Power-Set Encoding Function**
# def power_set_encoding(labels):
#     """Encodes multi-label speaker activation into a single integer using power-set encoding."""
#     return sum([l * (2 ** i) for i, l in enumerate(labels) if l in [0, 1]])

# # **Overlapping Speech Simulation**
# def simulate_overlap(grouped_data, num_speakers=2, duration=10, sampling_rate=16000):
#     overlapping_segments = []
#     labels = []
#     for meeting_id, samples in grouped_data.items():
#         speakers = {sample["speaker_id"]: [] for sample in samples}
#         for sample in samples:
#             speakers[sample["speaker_id"]].append(sample)
#         if len(speakers) < num_speakers:
#             continue
#         for _ in range(50):
#             chosen_speakers = random.sample(list(speakers.keys()), num_speakers)
#             combined_signal = None
#             speaker_label = [0] * len(speakers)
#             for speaker_id in chosen_speakers:
#                 sample = random.choice(speakers[speaker_id])
#                 signal = sample["audio"]["array"]
#                 if len(signal) > duration * sampling_rate:
#                     signal = signal[: duration * sampling_rate]
#                 else:
#                     signal = np.pad(signal, (0, duration * sampling_rate - len(signal)), mode="constant")
#                 if combined_signal is None:
#                     combined_signal = signal
#                 else:
#                     combined_signal += signal
#                 speaker_label[list(speakers.keys()).index(speaker_id)] = 1
#             if combined_signal is not None and np.max(np.abs(combined_signal)) > 0:
#                 combined_signal = combined_signal / np.max(np.abs(combined_signal))
#             else:
#                 continue
#             overlapping_segments.append(combined_signal)
#             labels.append(power_set_encoding(speaker_label))

#     # ✅ Debug: Print the first few labels to check if overlapping speech is simulated correctly
#     print("🔹 Sample labels from train set (Power-Set Encoded):")
#     for i in range(min(10, len(labels))):  # Print up to 10 samples
#         print(labels[i])

#     return overlapping_segments, labels


# # **Feature Extraction (Log-Mel Spectrograms)**
# def extract_features_batch(segments, sr=16000, n_mels=80, win_length=0.025, hop_length=0.01, max_frames=None):
#     features = []
#     win_length_samples = int(win_length * sr)
#     hop_length_samples = int(hop_length * sr)

#     for segment in segments:
#         mel_spec = librosa.feature.melspectrogram(
#             y=segment, sr=sr, n_mels=n_mels, win_length=win_length_samples, hop_length=hop_length_samples
#         )
#         log_mel = librosa.power_to_db(mel_spec)

#         if max_frames:
#             log_mel = log_mel[:, :max_frames] if log_mel.shape[1] > max_frames else np.pad(
#                 log_mel, ((0, 0), (0, max_frames - log_mel.shape[1])), mode="constant"
#             )

#         features.append(log_mel.T)

#     features = np.array(features, dtype=np.float32)

#     return torch.tensor(features, dtype=torch.float32)  # ✅ Corrected tensor conversion

# class OverlappingSpeechDataset(Dataset):
#     def __init__(self, features, labels):
#         self.features = features
#         self.labels = labels

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         feature = torch.tensor(self.features[idx], dtype=torch.float32)
#         label = torch.tensor(self.labels[idx], dtype=torch.long)

#         return feature, label

# scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Scaling

# # **Training Execution**
# num_speakers = 5

# # Simulate overlapping speech for training
# train_segments, train_labels = simulate_overlap(grouped_train)
# val_segments, val_labels = simulate_overlap(grouped_validation)
# test_segments, test_labels = simulate_overlap(grouped_test)

# from collections import Counter

# print("🔹 Unique Labels in Training Set:", set(train_labels))
# print("🔹 Label Distribution:")
# label_counts = Counter(train_labels)
# for label, count in label_counts.items():
#     binary_repr = bin(label)[2:].zfill(num_speakers)  # Convert to binary
#     print(f"Label: {label}, Binary: {binary_repr}, Count: {count}")

# unique_labels = sorted(set(train_labels + val_labels))  # ✅ Include val_labels
# label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
# index_to_label = {idx: label for label, idx in label_to_index.items()}

# print(f"🔹 Label Mapping: {label_to_index}")

# num_classes = len(unique_labels)  # ✅ Actual number of classes

# # Apply mapping to training and validation labels
# train_labels = [label_to_index[label] for label in train_labels]
# val_labels = [label_to_index[label] for label in val_labels]

# print(f"🔹 Mapped Training Labels: {train_labels[:10]}")  # Debugging

# from torch.nn.utils.rnn import pad_sequence

# def collate_fn_pad(batch):
#     """Pads variable-length sequences to the longest in the batch, ensuring consistent batch shapes."""
#     batch = [b for b in batch if b is not None]  # Remove None samples

#     if len(batch) == 0:
#         return None, None  # Return empty batch if all are invalid

#     features, labels = zip(*batch)

#     # ✅ Pad sequences to the maximum length in this batch
#     features_padded = pad_sequence(features, batch_first=True, padding_value=0)

#     # ✅ Keep labels as scalars (no expansion)
#     labels = torch.tensor(labels, dtype=torch.long)

#     return features_padded, labels


# max_frames = int((10 / 0.01))  # 10s duration, 0.01 hop length (10s / 10ms)
# train_features = extract_features_batch(train_segments, max_frames=max_frames)
# val_features = extract_features_batch(val_segments, max_frames=max_frames)
# test_features = extract_features_batch(test_segments, max_frames=max_frames)

# train_loader = DataLoader(
#     OverlappingSpeechDataset(train_features, train_labels),
#     batch_size=16,
#     shuffle=True,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# val_loader = DataLoader(
#     OverlappingSpeechDataset(val_features, val_labels),
#     batch_size=16,
#     shuffle=False,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# test_loader = DataLoader(
#     OverlappingSpeechDataset(test_features, test_labels),
#     batch_size=16,
#     shuffle=False,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # ✅ Define `speaker_encoder` correctly
# speaker_encoder = EncoderClassifier.from_hparams(
#     source="speechbrain/spkrec-ecapa-voxceleb",
#     savedir="pretrained_models/spkrec-ecapa",
#     run_opts={"device": DEVICE}
# ).to(DEVICE)

# # **SEND Model**
# class SpeechEncoder(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers):
#         super().__init__()
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)

#     def forward(self, x):
#         x, _ = self.lstm(x)
#         return x

# class SEND(nn.Module):
#     def __init__(self, input_dim, speaker_dim, hidden_dim, num_layers, num_speakers):
#         super().__init__()
#         self.speech_encoder = SpeechEncoder(input_dim, hidden_dim, num_layers)
#         self.speaker_encoder = speaker_encoder.encode_batch

#         self.feature_projection = nn.Linear(hidden_dim * 2, hidden_dim * 2)
#         self.context_independent = nn.Linear(hidden_dim * 2, num_speakers)
#         self.context_dependent = nn.TransformerEncoder(nn.TransformerEncoderLayer(hidden_dim * 2, 4), num_layers)
#         self.match_projection = nn.Linear(hidden_dim * 2, num_speakers)
#         self.post_net = nn.Linear(hidden_dim * 2, 2 ** num_speakers)  # Ensure input matches hidden_dim * 2

#     def forward(self, speech):
#         # Ensure correct shape
#         if speech.dim() == 4:
#             speech = speech.squeeze(1)

#         speech_features = self.speech_encoder(speech)
#         # print(f"🔹 Speech features shape after LSTM: {speech_features.shape}")  # [16, 29, 512]

#         speech_features = self.feature_projection(speech_features)
#         # print(f"🔹 Projected speech features shape: {speech_features.shape}")  # [16, 29, 512]

#         # **Apply mean pooling across time dimension (sequence length 29)**
#         pooled_features = torch.mean(speech_features, dim=1)  # [16, 512]
#         # print(f"🔹 Pooled features shape: {pooled_features.shape}")  # [16, 512]

#         # Ensure output matches batch size
#         output = self.post_net(pooled_features)  # [16, num_classes]
#         # print(f"🔹 Model output shape: {output.shape}")  # Should be [16, 2**num_speakers]

#         return output

# model = SEND(input_dim=80, speaker_dim=512, hidden_dim=256, num_layers=4, num_speakers=num_speakers).to(DEVICE)

# from pyannote.core import Annotation, Segment
# from pyannote.metrics.diarization import DiarizationErrorRate

# def compute_der(predictions, ground_truths, timestamps):
#     metric = DiarizationErrorRate()
#     pred_annotation = Annotation()
#     gt_annotation = Annotation()

#     for i, (start, end) in enumerate(timestamps):
#         pred_annotation[Segment(start, end)] = str(predictions[i])
#         gt_annotation[Segment(start, end)] = str(ground_truths[i])

#     der_score = metric(gt_annotation, pred_annotation)
#     logging.info(f"🔹 DER Computed: {der_score:.4f}")
#     return der_score

# def labels_to_annotation(labels, num_speakers, frame_duration=0.01):
#     annotation = Annotation()

#     if isinstance(labels, (np.ndarray, torch.Tensor)):
#         labels = labels.flatten().tolist()  # Convert tensor to list

#     label = int(mode(labels, keepdims=True)[0][0])  # ✅ Fixed mode extraction
#     print(f"🔹 Decoded Label: {label}")

#     active_speakers = [bool(label & (1 << i)) for i in range(num_speakers)]

#     if isinstance(labels, (list, np.ndarray, torch.Tensor)):
#         end_time = len(labels) * frame_duration
#     else:
#         end_time = frame_duration  # Single label case

#     start_time = 0
#     for speaker_idx, is_active in enumerate(active_speakers):
#         if is_active:
#             annotation[Segment(start_time, end_time)] = f"Speaker_{speaker_idx}"

#     print(f"Decoded label: {label}, Active speakers: {active_speakers}")

#     return annotation

# def calculate_der(model, test_loader, num_speakers):
#     der_metric = DiarizationErrorRate()
#     total_der = 0
#     num_samples = 0

#     with torch.no_grad():
#         for i, (features, labels) in enumerate(test_loader):
#             features = features.to(device)
#             outputs = model(features)
#             predicted_labels = torch.argmax(outputs, dim=-1).cpu().numpy()
#             labels = labels.cpu().numpy()

#             for j in range(len(labels)):
#                 ref_annotation = labels_to_annotation(labels[j], num_speakers)
#                 hyp_annotation = labels_to_annotation(predicted_labels[j], num_speakers)

#                 der = der_metric(ref_annotation, hyp_annotation)

#                 logging.info(f"🔹 Sample {i}-{j} | DER: {der:.2%}")

#                 total_der += der
#                 num_samples += 1

#     avg_der = total_der / num_samples if num_samples > 0 else 0
#     logging.info(f"\n✅ Final Validation DER: {avg_der:.2%}")
#     return avg_der


# class FLClient(NumPyClient):
#     def __init__(self, model, train_loader, test_loader):
#         os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # ✅ Force CPU globally
#         self.device = torch.device("cpu")  # ✅ Force CPU for PyTorch
#         logging.info(f"Using device: {self.device}")
#         self.model = model.to(self.device)  # ✅ Move model to CPU
#         self.train_loader = train_loader
#         self.test_loader = test_loader
#         self.optimizer = optim.Adam(model.parameters(), lr=0.001)
#         self.criterion = nn.CrossEntropyLoss()

#     def get_parameters(self, config=None):
#         return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

#     def set_parameters(self, parameters):
#         state_dict = {k: torch.tensor(v).to(self.device) for k, v in zip(self.model.state_dict().keys(), parameters)}
#         self.model.load_state_dict(state_dict)
#         self.model.to(self.device)  # Ensure model is on the right device

#     def fit(self, parameters, config):
#         self.set_parameters(parameters)
#         self.model.train()

#         logging.info(f"Training on {self.device}...")

#         # 🔹 Check initial weights before training
#         initial_weight = self.model.speech_encoder.lstm.weight_ih_l0.clone().detach().cpu()

#         total_loss = 0
#         for epoch in range(2):
#             for batch_idx, (features, labels) in enumerate(self.train_loader):
#                 features, labels = features.to(self.device), labels.to(self.device)

#                 logging.info(f"Batch {batch_idx} ")
#                 logging.info(f"Sample labels: {labels[:5]}")

#                 self.optimizer.zero_grad()
#                 outputs = self.model(features)

#                 preds = torch.argmax(outputs, dim=-1).cpu().tolist()  # 🔹 Get predictions

#                 labels = labels.view(-1)
#                 loss = self.criterion(outputs, labels.long())
#                 loss.backward()
#                 self.optimizer.step()

#                 total_loss += loss.item()

#                 # 🔹 Check if predictions vary across batches
#                 logging.info(f"🔹 Predictions (Batch {batch_idx}): {preds[:5]}")
#                 logging.info(f"🔹 Ground Truth: {labels[:5].tolist()}")



#         avg_loss = total_loss / len(self.train_loader)

#         # 🔹 Check if weights changed after training
#         updated_weight = self.model.speech_encoder.lstm.weight_ih_l0.clone().detach().cpu()
#         weight_change = torch.sum((updated_weight - initial_weight) ** 2).item()

#         logging.info(f"✅ Training completed. Avg Loss: {avg_loss:.4f}, Weight Change: {weight_change:.6f}")

#         return self.get_parameters(), len(self.train_loader.dataset), {"loss": avg_loss, "weight_change": weight_change}

#     def evaluate(self, parameters, config):
#         self.set_parameters(parameters)  # Load the latest model parameters
#         self.model.eval()

#         logging.info(f"🔹 Evaluating Model on {self.device}...")

#         avg_der = calculate_der(self.model, self.test_loader, num_speakers)

#         logging.info(f"✅ Evaluation Results - DER: {avg_der:.2%}")

#         return 0.0, len(self.test_loader.dataset), {"der": avg_der}


# # ✅ Custom Strategy for Evaluation Debugging
# class PrintEvaluateStrategy(fl.server.strategy.FedAvg):
#     def aggregate_evaluate(self, rnd: int, results, failures):
#         aggregated = super().aggregate_evaluate(rnd, results, failures)
#         if aggregated is None:
#             logging.info(f"[ROUND {rnd}] No evaluation results")
#             return aggregated
#         if isinstance(aggregated, tuple):
#             if len(aggregated) == 3:
#                 loss, num_examples, metrics = aggregated
#             elif len(aggregated) == 2:
#                 loss, num_examples = aggregated
#                 num_examples = num_examples if isinstance(num_examples, int) else "unknown"
#                 metrics = {}
#             else:
#                 loss, num_examples, metrics = aggregated, "unknown", {}
#         else:
#             logging.info(f"[ROUND {rnd}] Aggregated evaluation: {aggregated}")
#             return aggregated

#         accuracy = metrics.get("accuracy", None)
#         accuracy_str = f"{accuracy:.2%}" if accuracy is not None else "N/A"
#         logging.info(f"[ROUND {rnd}] Evaluation results: Loss: {loss:.4f}, Accuracy: {accuracy_str} on {num_examples} examples")
#         return aggregated

# logging.basicConfig(level=logging.INFO)
# strategy = PrintEvaluateStrategy(min_fit_clients=4, min_available_clients=4)

# for features, labels in train_loader:
#     print(f"Feature shape: {features.shape}, Label shape: {labels.shape}")
#     break  # Only check first batch


# fl.simulation.start_simulation(
#     client_fn=lambda ctx: FLClient(model, train_loader, test_loader),
#     num_clients=4,
#     config=fl.server.ServerConfig(num_rounds=2),
#     strategy=fl.server.strategy.FedAvg(),
#     ray_init_args={
#         "num_cpus": 4,  # ✅ Only use CPU
#         "include_dashboard": False,
#         "ignore_reinit_error": True,
#     },
#     client_resources={"num_cpus": 1}  # ✅ No GPU needed
# )

## batch=1

In [9]:
# import logging
# import time
# import flwr as fl

# import importlib
# import ray
# importlib.reload(ray)

# from speechbrain.pretrained import EncoderClassifier

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import librosa
# import numpy as np
# import random
# from torch.utils.data import Dataset, DataLoader
# from speechbrain.pretrained import EncoderClassifier
# from torch.optim import Adam
# from pyannote.metrics.diarization import DiarizationErrorRate
# from scipy.stats import mode

# torch.cuda.empty_cache()

# # **Set device**
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # **Load Pretrained Speaker Encoder (ECAPA-TDNN)**
# speaker_encoder = EncoderClassifier.from_hparams(
#     source="speechbrain/spkrec-ecapa-voxceleb",
#     savedir="pretrained_models/spkrec-ecapa",
#     run_opts={"device": device}
# ).to(device)

# # **Freeze the speaker encoder**
# for param in speaker_encoder.parameters():
#     param.requires_grad = False

# # **Function to extract speaker embeddings**
# def extract_speaker_embedding(features):
#     """Extracts speaker embeddings using ECAPA-TDNN."""
#     with torch.no_grad():
#         embeddings = speaker_encoder.encode_batch(features)
#         return embeddings.squeeze(1)  # Remove unnecessary dimension

# # **Power-Set Encoding Function**
# def power_set_encoding(labels):
#     """Encodes multi-label speaker activation into a single integer using power-set encoding."""
#     return sum([l * (2 ** i) for i, l in enumerate(labels) if l in [0, 1]])

# # **Overlapping Speech Simulation**
# def simulate_overlap(grouped_data, num_speakers=2, duration=10, sampling_rate=16000):
#     overlapping_segments = []
#     labels = []
#     for meeting_id, samples in grouped_data.items():
#         speakers = {sample["speaker_id"]: [] for sample in samples}
#         for sample in samples:
#             speakers[sample["speaker_id"]].append(sample)
#         if len(speakers) < num_speakers:
#             continue
#         for _ in range(50):
#             chosen_speakers = random.sample(list(speakers.keys()), num_speakers)
#             combined_signal = None
#             speaker_label = [0] * len(speakers)
#             for speaker_id in chosen_speakers:
#                 sample = random.choice(speakers[speaker_id])
#                 signal = sample["audio"]["array"]
#                 if len(signal) > duration * sampling_rate:
#                     signal = signal[: duration * sampling_rate]
#                 else:
#                     signal = np.pad(signal, (0, duration * sampling_rate - len(signal)), mode="constant")
#                 if combined_signal is None:
#                     combined_signal = signal
#                 else:
#                     combined_signal += signal
#                 speaker_label[list(speakers.keys()).index(speaker_id)] = 1
#             if combined_signal is not None and np.max(np.abs(combined_signal)) > 0:
#                 combined_signal = combined_signal / np.max(np.abs(combined_signal))
#             else:
#                 continue
#             overlapping_segments.append(combined_signal)
#             labels.append(power_set_encoding(speaker_label))

#     # ✅ Debug: Print the first few labels to check if overlapping speech is simulated correctly
#     print("🔹 Sample labels from train set (Power-Set Encoded):")
#     for i in range(min(10, len(labels))):  # Print up to 10 samples
#         print(labels[i])

#     return overlapping_segments, labels


# # **Feature Extraction (Log-Mel Spectrograms)**
# def extract_features_batch(segments, sr=16000, n_mels=80, win_length=0.025, hop_length=0.01, max_frames=None):
#     features = []
#     win_length_samples = int(win_length * sr)
#     hop_length_samples = int(hop_length * sr)

#     for segment in segments:
#         mel_spec = librosa.feature.melspectrogram(
#             y=segment, sr=sr, n_mels=n_mels, win_length=win_length_samples, hop_length=hop_length_samples
#         )
#         log_mel = librosa.power_to_db(mel_spec)

#         if max_frames:
#             log_mel = log_mel[:, :max_frames] if log_mel.shape[1] > max_frames else np.pad(
#                 log_mel, ((0, 0), (0, max_frames - log_mel.shape[1])), mode="constant"
#             )

#         features.append(log_mel.T)

#     features = np.array(features, dtype=np.float32)

#     return torch.tensor(features, dtype=torch.float32)  # ✅ Corrected tensor conversion

# class OverlappingSpeechDataset(Dataset):
#     def __init__(self, features, labels):
#         self.features = features
#         self.labels = labels

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         feature = torch.tensor(self.features[idx], dtype=torch.float32)
#         label = torch.tensor(self.labels[idx], dtype=torch.long)

#         return feature, label

# scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Scaling

# # **Training Execution**
# num_speakers = 5

# # Simulate overlapping speech for training
# train_segments, train_labels = simulate_overlap(grouped_train)
# val_segments, val_labels = simulate_overlap(grouped_validation)
# test_segments, test_labels = simulate_overlap(grouped_test)

# from collections import Counter

# print("🔹 Unique Labels in Training Set:", set(train_labels))
# print("🔹 Label Distribution:")
# label_counts = Counter(train_labels)
# for label, count in label_counts.items():
#     binary_repr = bin(label)[2:].zfill(num_speakers)  # Convert to binary
#     print(f"Label: {label}, Binary: {binary_repr}, Count: {count}")

# unique_labels = sorted(set(train_labels + val_labels))  # ✅ Include val_labels
# label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
# index_to_label = {idx: label for label, idx in label_to_index.items()}

# print(f"🔹 Label Mapping: {label_to_index}")

# num_classes = len(unique_labels)  # ✅ Actual number of classes

# # Apply mapping to training and validation labels
# train_labels = [label_to_index[label] for label in train_labels]
# val_labels = [label_to_index[label] for label in val_labels]

# print(f"🔹 Mapped Training Labels: {train_labels[:10]}")  # Debugging

# from torch.nn.utils.rnn import pad_sequence

# def collate_fn_pad(batch):
#     """Pads variable-length sequences to the longest in the batch, ensuring consistent batch shapes."""
#     batch = [b for b in batch if b is not None]  # Remove None samples

#     if len(batch) == 0:
#         return None, None  # Return empty batch if all are invalid

#     features, labels = zip(*batch)

#     # ✅ Pad sequences to the maximum length in this batch
#     features_padded = pad_sequence(features, batch_first=True, padding_value=0)

#     # ✅ Keep labels as scalars (no expansion)
#     labels = torch.tensor(labels, dtype=torch.long)

#     return features_padded, labels


# max_frames = int((10 / 0.01))  # 10s duration, 0.01 hop length (10s / 10ms)
# train_features = extract_features_batch(train_segments, max_frames=max_frames)
# val_features = extract_features_batch(val_segments, max_frames=max_frames)
# test_features = extract_features_batch(test_segments, max_frames=max_frames)

# train_loader = DataLoader(
#     OverlappingSpeechDataset(train_features, train_labels),
#     batch_size=1,
#     shuffle=True,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# val_loader = DataLoader(
#     OverlappingSpeechDataset(val_features, val_labels),
#     batch_size=1,
#     shuffle=False,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# test_loader = DataLoader(
#     OverlappingSpeechDataset(test_features, test_labels),
#     batch_size=1,
#     shuffle=False,
#     collate_fn=collate_fn_pad  # ✅ Apply collate function
# )

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # ✅ Define `speaker_encoder` correctly
# speaker_encoder = EncoderClassifier.from_hparams(
#     source="speechbrain/spkrec-ecapa-voxceleb",
#     savedir="pretrained_models/spkrec-ecapa",
#     run_opts={"device": DEVICE}
# ).to(DEVICE)

# # **SEND Model**
# class SpeechEncoder(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers):
#         super().__init__()
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)

#     def forward(self, x):
#         x, _ = self.lstm(x)
#         return x

# class SEND(nn.Module):
#     def __init__(self, input_dim, speaker_dim, hidden_dim, num_layers, num_speakers):
#         super().__init__()
#         self.speech_encoder = SpeechEncoder(input_dim, hidden_dim, num_layers)
#         self.speaker_encoder = speaker_encoder.encode_batch

#         self.feature_projection = nn.Linear(hidden_dim * 2, hidden_dim * 2)
#         self.context_independent = nn.Linear(hidden_dim * 2, num_speakers)
#         self.context_dependent = nn.TransformerEncoder(nn.TransformerEncoderLayer(hidden_dim * 2, 4), num_layers)
#         self.match_projection = nn.Linear(hidden_dim * 2, num_speakers)
#         self.post_net = nn.Linear(hidden_dim * 2, 2 ** num_speakers)  # Ensure input matches hidden_dim * 2

#     def forward(self, speech):
#         # Ensure correct shape
#         if speech.dim() == 4:
#             speech = speech.squeeze(1)

#         speech_features = self.speech_encoder(speech)
#         # print(f"🔹 Speech features shape after LSTM: {speech_features.shape}")  # [16, 29, 512]

#         speech_features = self.feature_projection(speech_features)
#         # print(f"🔹 Projected speech features shape: {speech_features.shape}")  # [16, 29, 512]

#         # **Apply mean pooling across time dimension (sequence length 29)**
#         pooled_features = torch.mean(speech_features, dim=1)  # [16, 512]
#         # print(f"🔹 Pooled features shape: {pooled_features.shape}")  # [16, 512]

#         # Ensure output matches batch size
#         output = self.post_net(pooled_features)  # [16, num_classes]
#         # print(f"🔹 Model output shape: {output.shape}")  # Should be [16, 2**num_speakers]

#         return output

# model = SEND(input_dim=80, speaker_dim=512, hidden_dim=256, num_layers=4, num_speakers=num_speakers).to(DEVICE)

# from pyannote.core import Annotation, Segment
# from pyannote.metrics.diarization import DiarizationErrorRate

# def compute_der(predictions, ground_truths, timestamps):
#     metric = DiarizationErrorRate()
#     pred_annotation = Annotation()
#     gt_annotation = Annotation()

#     for i, (start, end) in enumerate(timestamps):
#         pred_annotation[Segment(start, end)] = str(predictions[i])
#         gt_annotation[Segment(start, end)] = str(ground_truths[i])

#     der_score = metric(gt_annotation, pred_annotation)
#     logging.info(f"🔹 DER Computed: {der_score:.4f}")
#     return der_score

# def labels_to_annotation(labels, num_speakers, frame_duration=0.01):
#     annotation = Annotation()

#     if isinstance(labels, (np.ndarray, torch.Tensor)):
#         labels = labels.flatten().tolist()  # Convert tensor to list

#     label = int(mode(labels, keepdims=True)[0][0])  # ✅ Fixed mode extraction
#     print(f"🔹 Decoded Label: {label}")

#     active_speakers = [bool(label & (1 << i)) for i in range(num_speakers)]

#     if isinstance(labels, (list, np.ndarray, torch.Tensor)):
#         end_time = len(labels) * frame_duration
#     else:
#         end_time = frame_duration  # Single label case

#     start_time = 0
#     for speaker_idx, is_active in enumerate(active_speakers):
#         if is_active:
#             annotation[Segment(start_time, end_time)] = f"Speaker_{speaker_idx}"

#     print(f"Decoded label: {label}, Active speakers: {active_speakers}")

#     return annotation

# def calculate_der(model, test_loader, num_speakers):
#     der_metric = DiarizationErrorRate()
#     total_der = 0
#     num_samples = 0

#     with torch.no_grad():
#         for i, (features, labels) in enumerate(test_loader):
#             features = features.to(device)
#             outputs = model(features)
#             predicted_labels = torch.argmax(outputs, dim=-1).cpu().numpy()
#             labels = labels.cpu().numpy()

#             for j in range(len(labels)):
#                 ref_annotation = labels_to_annotation(labels[j], num_speakers)
#                 hyp_annotation = labels_to_annotation(predicted_labels[j], num_speakers)

#                 der = der_metric(ref_annotation, hyp_annotation)

#                 logging.info(f"🔹 Sample {i}-{j} | DER: {der:.2%}")

#                 total_der += der
#                 num_samples += 1

#     avg_der = total_der / num_samples if num_samples > 0 else 0
#     logging.info(f"\n✅ Final Validation DER: {avg_der:.2%}")
#     return avg_der


# class FLClient(NumPyClient):
#     def __init__(self, model, train_loader, test_loader):
#         os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # ✅ Force CPU globally
#         self.device = torch.device("cpu")  # ✅ Force CPU for PyTorch
#         logging.info(f"Using device: {self.device}")
#         self.model = model.to(self.device)  # ✅ Move model to CPU
#         self.train_loader = train_loader
#         self.test_loader = test_loader
#         self.optimizer = optim.Adam(model.parameters(), lr=0.001)
#         self.criterion = nn.CrossEntropyLoss()

#     def get_parameters(self, config=None):
#         return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

#     def set_parameters(self, parameters):
#         state_dict = {k: torch.tensor(v).to(self.device) for k, v in zip(self.model.state_dict().keys(), parameters)}
#         self.model.load_state_dict(state_dict)
#         self.model.to(self.device)  # Ensure model is on the right device

#     def fit(self, parameters, config):
#         self.set_parameters(parameters)
#         self.model.train()

#         logging.info(f"Training on {self.device}...")

#         # 🔹 Check initial weights before training
#         initial_weight = self.model.speech_encoder.lstm.weight_ih_l0.clone().detach().cpu()

#         total_loss = 0
#         for epoch in range(2):
#             for batch_idx, (features, labels) in enumerate(self.train_loader):
#                 features, labels = features.to(self.device), labels.to(self.device)

#                 logging.info(f"Batch {batch_idx} ")
#                 logging.info(f"Sample labels: {labels[:5]}")

#                 self.optimizer.zero_grad()
#                 outputs = self.model(features)

#                 preds = torch.argmax(outputs, dim=-1).cpu().tolist()  # 🔹 Get predictions

#                 labels = labels.view(-1)
#                 loss = self.criterion(outputs, labels.long())
#                 loss.backward()
#                 self.optimizer.step()

#                 total_loss += loss.item()

#                 # 🔹 Check if predictions vary across batches
#                 logging.info(f"🔹 Predictions (Batch {batch_idx}): {preds[:5]}")
#                 logging.info(f"🔹 Ground Truth: {labels[:5].tolist()}")



#         avg_loss = total_loss / len(self.train_loader)

#         # 🔹 Check if weights changed after training
#         updated_weight = self.model.speech_encoder.lstm.weight_ih_l0.clone().detach().cpu()
#         weight_change = torch.sum((updated_weight - initial_weight) ** 2).item()

#         logging.info(f"✅ Training completed. Avg Loss: {avg_loss:.4f}, Weight Change: {weight_change:.6f}")

#         return self.get_parameters(), len(self.train_loader.dataset), {"loss": avg_loss, "weight_change": weight_change}

#     def evaluate(self, parameters, config):
#         self.set_parameters(parameters)  # Load the latest model parameters
#         self.model.eval()

#         logging.info(f"🔹 Evaluating Model on {self.device}...")

#         avg_der = calculate_der(self.model, self.test_loader, num_speakers)

#         logging.info(f"✅ Evaluation Results - DER: {avg_der:.2%}")

#         return 0.0, len(self.test_loader.dataset), {"der": avg_der}


# # ✅ Custom Strategy for Evaluation Debugging
# class PrintEvaluateStrategy(fl.server.strategy.FedAvg):
#     def aggregate_evaluate(self, rnd: int, results, failures):
#         aggregated = super().aggregate_evaluate(rnd, results, failures)
#         if aggregated is None:
#             logging.info(f"[ROUND {rnd}] No evaluation results")
#             return aggregated
#         if isinstance(aggregated, tuple):
#             if len(aggregated) == 3:
#                 loss, num_examples, metrics = aggregated
#             elif len(aggregated) == 2:
#                 loss, num_examples = aggregated
#                 num_examples = num_examples if isinstance(num_examples, int) else "unknown"
#                 metrics = {}
#             else:
#                 loss, num_examples, metrics = aggregated, "unknown", {}
#         else:
#             logging.info(f"[ROUND {rnd}] Aggregated evaluation: {aggregated}")
#             return aggregated

#         accuracy = metrics.get("accuracy", None)
#         accuracy_str = f"{accuracy:.2%}" if accuracy is not None else "N/A"
#         logging.info(f"[ROUND {rnd}] Evaluation results: Loss: {loss:.4f}, Accuracy: {accuracy_str} on {num_examples} examples")
#         return aggregated

# logging.basicConfig(level=logging.INFO)
# strategy = PrintEvaluateStrategy(min_fit_clients=4, min_available_clients=4)

# for features, labels in train_loader:
#     print(f"Feature shape: {features.shape}, Label shape: {labels.shape}")
#     break  # Only check first batch


# fl.simulation.start_simulation(
#     client_fn=lambda ctx: FLClient(model, train_loader, test_loader),
#     num_clients=4,
#     config=fl.server.ServerConfig(num_rounds=2),
#     strategy=fl.server.strategy.FedAvg(),
#     ray_init_args={
#         "num_cpus": 4,  # ✅ Only use CPU
#         "include_dashboard": False,
#         "ignore_reinit_error": True,
#     },
#     client_resources={"num_cpus": 1}  # ✅ No GPU needed
# )

## Batch = 1, fixing DER=0

In [10]:
import logging
import time
import flwr as fl

import importlib
import ray
importlib.reload(ray)

from speechbrain.pretrained import EncoderClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
from speechbrain.pretrained import EncoderClassifier
from torch.optim import Adam
from pyannote.metrics.diarization import DiarizationErrorRate
from scipy.stats import mode

torch.cuda.empty_cache()

# **Set device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Load Pretrained Speaker Encoder (ECAPA-TDNN)**
speaker_encoder = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_models/spkrec-ecapa",
    run_opts={"device": device}
).to(device)

# **Freeze the speaker encoder**
for param in speaker_encoder.parameters():
    param.requires_grad = False

# **Function to extract speaker embeddings**
def extract_speaker_embedding(features):
    """Extracts speaker embeddings using ECAPA-TDNN."""
    with torch.no_grad():
        embeddings = speaker_encoder.encode_batch(features)
        return embeddings.squeeze(1)  # Remove unnecessary dimension

# **Power-Set Encoding Function**
def power_set_encoding(labels):
    """Encodes multi-label speaker activation into a single integer using power-set encoding."""
    return sum([l * (2 ** i) for i, l in enumerate(labels) if l in [0, 1]])

# **Overlapping Speech Simulation**
def simulate_overlap(grouped_data, num_speakers=2, duration=10, sampling_rate=16000):
    overlapping_segments = []
    labels = []
    for meeting_id, samples in grouped_data.items():
        speakers = {sample["speaker_id"]: [] for sample in samples}
        for sample in samples:
            speakers[sample["speaker_id"]].append(sample)
        if len(speakers) < num_speakers:
            continue
        for _ in range(50):
            chosen_speakers = random.sample(list(speakers.keys()), num_speakers)
            combined_signal = None
            speaker_label = [0] * len(speakers)
            for speaker_id in chosen_speakers:
                sample = random.choice(speakers[speaker_id])
                signal = sample["audio"]["array"]
                if len(signal) > duration * sampling_rate:
                    signal = signal[: duration * sampling_rate]
                else:
                    signal = np.pad(signal, (0, duration * sampling_rate - len(signal)), mode="constant")
                if combined_signal is None:
                    combined_signal = signal
                else:
                    combined_signal += signal
                speaker_label[list(speakers.keys()).index(speaker_id)] = 1
            if combined_signal is not None and np.max(np.abs(combined_signal)) > 0:
                combined_signal = combined_signal / np.max(np.abs(combined_signal))
            else:
                continue
            overlapping_segments.append(combined_signal)
            labels.append(power_set_encoding(speaker_label))

    # ✅ Debug: Print the first few labels to check if overlapping speech is simulated correctly
    print("🔹 Sample labels from train set (Power-Set Encoded):")
    for i in range(min(10, len(labels))):  # Print up to 10 samples
        print(labels[i])

    return overlapping_segments, labels


# **Feature Extraction (Log-Mel Spectrograms)**
def extract_features_batch(segments, sr=16000, n_mels=80, win_length=0.025, hop_length=0.01, max_frames=None):
    features = []
    win_length_samples = int(win_length * sr)
    hop_length_samples = int(hop_length * sr)

    for segment in segments:
        mel_spec = librosa.feature.melspectrogram(
            y=segment, sr=sr, n_mels=n_mels, win_length=win_length_samples, hop_length=hop_length_samples
        )
        log_mel = librosa.power_to_db(mel_spec)

        if max_frames:
            log_mel = log_mel[:, :max_frames] if log_mel.shape[1] > max_frames else np.pad(
                log_mel, ((0, 0), (0, max_frames - log_mel.shape[1])), mode="constant"
            )

        features.append(log_mel.T)

    features = np.array(features, dtype=np.float32)

    return torch.tensor(features, dtype=torch.float32)  # ✅ Corrected tensor conversion

class OverlappingSpeechDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        return feature, label

scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Scaling

# **Training Execution**
num_speakers = 5

# Simulate overlapping speech for training
train_segments, train_labels = simulate_overlap(grouped_train)
val_segments, val_labels = simulate_overlap(grouped_validation)
test_segments, test_labels = simulate_overlap(grouped_test)

from collections import Counter

print("🔹 Unique Labels in Training Set:", set(train_labels))
print("🔹 Label Distribution:")
label_counts = Counter(train_labels)
for label, count in label_counts.items():
    binary_repr = bin(label)[2:].zfill(num_speakers)  # Convert to binary
    print(f"Label: {label}, Binary: {binary_repr}, Count: {count}")

unique_labels = sorted(set(train_labels + val_labels))  # ✅ Include val_labels
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
index_to_label = {idx: label for label, idx in label_to_index.items()}

print(f"🔹 Label Mapping: {label_to_index}")

num_classes = len(unique_labels)  # ✅ Actual number of classes

# Apply mapping to training and validation labels
train_labels = [label_to_index[label] for label in train_labels]
val_labels = [label_to_index[label] for label in val_labels]

print(f"🔹 Mapped Training Labels: {train_labels[:10]}")  # Debugging

from torch.nn.utils.rnn import pad_sequence

def collate_fn_pad(batch):
    """Pads variable-length sequences to the longest in the batch, ensuring consistent batch shapes."""
    batch = [b for b in batch if b is not None]  # Remove None samples

    if len(batch) == 0:
        return None, None  # Return empty batch if all are invalid

    features, labels = zip(*batch)

    # ✅ Pad sequences to the maximum length in this batch
    features_padded = pad_sequence(features, batch_first=True, padding_value=0)

    # ✅ Keep labels as scalars (no expansion)
    labels = torch.tensor(labels, dtype=torch.long)

    return features_padded, labels


max_frames = int((10 / 0.01))  # 10s duration, 0.01 hop length (10s / 10ms)
train_features = extract_features_batch(train_segments, max_frames=max_frames)
val_features = extract_features_batch(val_segments, max_frames=max_frames)
test_features = extract_features_batch(test_segments, max_frames=max_frames)

train_loader = DataLoader(
    OverlappingSpeechDataset(train_features, train_labels),
    batch_size=1,
    shuffle=True,
    collate_fn=collate_fn_pad  # ✅ Apply collate function
)

val_loader = DataLoader(
    OverlappingSpeechDataset(val_features, val_labels),
    batch_size=1,
    shuffle=False,
    collate_fn=collate_fn_pad  # ✅ Apply collate function
)

test_loader = DataLoader(
    OverlappingSpeechDataset(test_features, test_labels),
    batch_size=1,
    shuffle=False,
    collate_fn=collate_fn_pad  # ✅ Apply collate function
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Define `speaker_encoder` correctly
speaker_encoder = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_models/spkrec-ecapa",
    run_opts={"device": DEVICE}
).to(DEVICE)

# **SEND Model**
class SpeechEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)

    def forward(self, x):
        x, _ = self.lstm(x)
        return x

class SEND(nn.Module):
    def __init__(self, input_dim, speaker_dim, hidden_dim, num_layers, num_speakers):
        super().__init__()
        self.speech_encoder = SpeechEncoder(input_dim, hidden_dim, num_layers)
        self.speaker_encoder = speaker_encoder.encode_batch

        self.feature_projection = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.context_independent = nn.Linear(hidden_dim * 2, num_speakers)
        self.context_dependent = nn.TransformerEncoder(nn.TransformerEncoderLayer(hidden_dim * 2, 4), num_layers)
        self.match_projection = nn.Linear(hidden_dim * 2, num_speakers)
        self.post_net = nn.Linear(hidden_dim * 2, 2 ** num_speakers)  # Ensure input matches hidden_dim * 2

    def forward(self, speech):
        # Ensure correct shape
        if speech.dim() == 4:
            speech = speech.squeeze(1)

        speech_features = self.speech_encoder(speech)
        # print(f"🔹 Speech features shape after LSTM: {speech_features.shape}")  # [16, 29, 512]

        speech_features = self.feature_projection(speech_features)
        # print(f"🔹 Projected speech features shape: {speech_features.shape}")  # [16, 29, 512]

        # **Apply mean pooling across time dimension (sequence length 29)**
        pooled_features = torch.mean(speech_features, dim=1)  # [16, 512]
        # print(f"🔹 Pooled features shape: {pooled_features.shape}")  # [16, 512]

        # Ensure output matches batch size
        output = self.post_net(pooled_features)  # [16, num_classes]
        # print(f"🔹 Model output shape: {output.shape}")  # Should be [16, 2**num_speakers]

        return output

model = SEND(input_dim=80, speaker_dim=512, hidden_dim=256, num_layers=4, num_speakers=num_speakers).to(DEVICE)

from pyannote.core import Annotation, Segment
from pyannote.metrics.diarization import DiarizationErrorRate

def compute_der(predictions, ground_truths, timestamps):
    metric = DiarizationErrorRate()
    pred_annotation = Annotation()
    gt_annotation = Annotation()

    for i, (start, end) in enumerate(timestamps):
        pred_annotation[Segment(start, end)] = str(predictions[i])
        gt_annotation[Segment(start, end)] = str(ground_truths[i])

    der_score = metric(gt_annotation, pred_annotation)
    logging.info(f"🔹 DER Computed: {der_score:.4f}")
    return der_score

def labels_to_annotation(labels, num_speakers, frame_duration=0.01):
    annotation = Annotation()

    if isinstance(labels, (np.ndarray, torch.Tensor)):
        labels = labels.flatten().tolist()  # Convert tensor to list

    label = int(mode(labels, keepdims=True)[0][0])  # ✅ Fixed mode extraction
    print(f"🔹 Decoded Label: {label}")

    active_speakers = [bool(label & (1 << i)) for i in range(num_speakers)]

    if isinstance(labels, (list, np.ndarray, torch.Tensor)):
        end_time = len(labels) * frame_duration
    else:
        end_time = frame_duration  # Single label case

    start_time = 0
    for speaker_idx, is_active in enumerate(active_speakers):
        if is_active:
            annotation[Segment(start_time, end_time)] = f"Speaker_{speaker_idx}"

    print(f"Decoded label: {label}, Active speakers: {active_speakers}")

    return annotation

def calculate_der(model, test_loader, num_speakers):
    der_metric = DiarizationErrorRate()
    total_der = 0
    num_samples = 0

    with torch.no_grad():
        for i, (features, labels) in enumerate(test_loader):
            features = features.to(device)
            outputs = model(features)
            predicted_labels = torch.argmax(outputs, dim=-1).cpu().numpy()
            labels = labels.cpu().numpy()

            for j in range(len(labels)):
                ref_annotation = labels_to_annotation(labels[j], num_speakers)
                hyp_annotation = labels_to_annotation(predicted_labels[j], num_speakers)

                der = der_metric(ref_annotation, hyp_annotation)

                logging.info(f"🔹 Sample {i}-{j} | DER: {der:.2%}")

                total_der += der
                num_samples += 1

    avg_der = total_der / num_samples if num_samples > 0 else 0
    logging.info(f"\n✅ Final Validation DER: {avg_der:.2%}")
    return avg_der


class FLClient(NumPyClient):
    def __init__(self, model, train_loader, test_loader):
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # ✅ Force CPU globally
        self.device = torch.device("cpu")  # ✅ Force CPU for PyTorch
        logging.info(f"Using device: {self.device}")
        self.model = model.to(self.device)  # ✅ Move model to CPU
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.criterion = nn.CrossEntropyLoss()

    def get_parameters(self, config=None):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        state_dict = {k: torch.tensor(v).to(self.device) for k, v in zip(self.model.state_dict().keys(), parameters)}
        self.model.load_state_dict(state_dict)
        self.model.to(self.device)  # Ensure model is on the right device

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()

        logging.info(f"Training on {self.device}...")

        # 🔹 Check initial weights before training
        initial_weight = self.model.speech_encoder.lstm.weight_ih_l0.clone().detach().cpu()

        total_loss = 0
        for epoch in range(1):
            for batch_idx, (features, labels) in enumerate(self.train_loader):
                features, labels = features.to(self.device), labels.to(self.device)

                logging.info(f"Batch {batch_idx} ")
                logging.info(f"Sample labels: {labels[:5]}")

                self.optimizer.zero_grad()
                outputs = self.model(features)

                preds = torch.argmax(outputs, dim=-1).cpu().tolist()  # 🔹 Get predictions

                labels = labels.view(-1)
                loss = self.criterion(outputs, labels.long())

                print(f"🔹 Batch {batch_idx} Loss: {loss.item()}")  # ✅ Check if loss is meaningful

                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

                # 🔹 Check if predictions vary across batches
                logging.info(f"🔹 Predictions (Batch {batch_idx}): {preds[:5]}")
                logging.info(f"🔹 Ground Truth: {labels[:5].tolist()}")



        avg_loss = total_loss / len(self.train_loader)

        # 🔹 Check if weights changed after training
        updated_weight = self.model.speech_encoder.lstm.weight_ih_l0.clone().detach().cpu()
        weight_change = torch.sum((updated_weight - initial_weight) ** 2).item()

        logging.info(f"✅ Training completed. Avg Loss: {avg_loss:.4f}, Weight Change: {weight_change:.6f}")

        return self.get_parameters(), len(self.train_loader.dataset), {"loss": avg_loss, "weight_change": weight_change}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)  # Load the latest model parameters
        self.model.eval()

        logging.info(f"🔹 Evaluating Model on {self.device}...")

        avg_der = calculate_der(self.model, self.test_loader, num_speakers)

        logging.info(f"✅ Evaluation Results - DER: {avg_der:.2%}")

        return 0.0, len(self.test_loader.dataset), {"der": avg_der}


# ✅ Custom Strategy for Evaluation Debugging
class PrintEvaluateStrategy(fl.server.strategy.FedAvg):
    def aggregate_evaluate(self, rnd: int, results, failures):
        aggregated = super().aggregate_evaluate(rnd, results, failures)
        if aggregated is None:
            logging.info(f"[ROUND {rnd}] No evaluation results")
            return aggregated
        if isinstance(aggregated, tuple):
            if len(aggregated) == 3:
                loss, num_examples, metrics = aggregated
            elif len(aggregated) == 2:
                loss, num_examples = aggregated
                num_examples = num_examples if isinstance(num_examples, int) else "unknown"
                metrics = {}
            else:
                loss, num_examples, metrics = aggregated, "unknown", {}
        else:
            logging.info(f"[ROUND {rnd}] Aggregated evaluation: {aggregated}")
            return aggregated

        accuracy = metrics.get("accuracy", None)
        accuracy_str = f"{accuracy:.2%}" if accuracy is not None else "N/A"
        logging.info(f"[ROUND {rnd}] Evaluation results: Loss: {loss:.4f}, Accuracy: {accuracy_str} on {num_examples} examples")
        return aggregated

logging.basicConfig(level=logging.INFO)
strategy = PrintEvaluateStrategy(min_fit_clients=4, min_available_clients=4)

for features, labels in train_loader:
    print(f"Feature shape: {features.shape}, Label shape: {labels.shape}")
    break  # Only check first batch


fl.simulation.start_simulation(
    client_fn=lambda ctx: FLClient(model, train_loader, test_loader),
    num_clients=4,
    config=fl.server.ServerConfig(num_rounds=2),
    strategy=fl.server.strategy.FedAvg(),
    ray_init_args={
        "num_cpus": 4,  # ✅ Only use CPU
        "include_dashboard": False,
        "ignore_reinit_error": True,
    },
    client_resources={"num_cpus": 1}  # ✅ No GPU needed
)

/usr/local/lib/python3.11/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

🔹 Sample labels from train set (Power-Set Encoded):
10
3
3
12
20
17
9
18
20
6
🔹 Sample labels from train set (Power-Set Encoded):
3
6
3
12
5
3
10
3
3
3
🔹 Sample labels from train set (Power-Set Encoded):
3
6
12
10
12
12
12
12
6
5
🔹 Unique Labels in Training Set: {3, 5, 6, 9, 10, 12, 17, 18, 20, 24}
🔹 Label Distribution:
Label: 10, Binary: 01010, Count: 32
Label: 3, Binary: 00011, Count: 21
Label: 12, Binary: 01100, Count: 23
Label: 20, Binary: 10100, Count: 14
Label: 17, Binary: 10001, Count: 14
Label: 9, Binary: 01001, Count: 25
Label: 18, Binary: 10010, Count: 12
Label: 6, Binary: 00110, Count: 20
Label: 24, Binary: 11000, Count: 15
Label: 5, Binary: 00101, Count: 24
🔹 Label Mapping: {3: 0, 5: 1, 6: 2, 9: 3, 10: 4, 12: 5, 17: 6, 18: 7, 20: 8, 24: 9}
🔹 Mapped Training Labels: [4, 0, 0, 5, 8, 6, 3, 7, 8, 2]


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
<ipython-input-10-272fc8c28c40>:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feature = torch.tensor(self.features[idx], dtype=torch.float32)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr ru

Feature shape: torch.Size([1, 1000, 80]), Label shape: torch.Size([1])


2025-02-28 10:53:08,853	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 4.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 32625102030.0, 'object_store_memory': 16312551014.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 4 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=11998) 2025-02-28 10:53:11.204695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=11998) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=11998) E0000 00:00:1740739991.249493   11998 cuda_dnn.cc:83

...

(ClientAppActor pid=11999) Decoded label: 12, Active speakers: [False, False, True, True, False] [repeated 3x across cluster]
(ClientAppActor pid=12000) 🔹 Decoded Label: 4 [repeated 72x across cluster]
(ClientAppActor pid=12000) Decoded label: 4, Active speakers: [False, False, True, False, False] [repeated 36x across cluster]
(ClientAppActor pid=11998) Decoded label: 10, Active speakers: [False, True, False, True, False] [repeated 8x across cluster]
(ClientAppActor pid=12000) Decoded label: 6, Active speakers: [False, True, True, False, False] [repeated 13x across cluster]
(ClientAppActor pid=11998) Decoded label: 9, Active speakers: [True, False, False, True, False] [repeated 7x across cluster]
(ClientAppActor pid=11999) Decoded label: 3, Active speakers: [True, True, False, False, False] [repeated 3x across cluster]


(ClientAppActor pid=12000) INFO:root:
(ClientAppActor pid=12000) ✅ Final Validation DER: 0.00%
(ClientAppActor pid=12000) INFO:root:✅ Evaluation Results - DER: 0.00%


(ClientAppActor pid=11999) Decoded label: 5, Active speakers: [True, False, True, False, False] [repeated 3x across cluster]


INFO :      aggregate_evaluate: received 4 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 1922.50s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.0
INFO :      		round 2: 0.0
INFO :      


History (loss, distributed):
	round 1: 0.0
	round 2: 0.0